In [ ]:
# https://towardsdatascience.com/binary-image-classification-in-pytorch-5adf64f8c781
# https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-k-fold-cross-validation-with-pytorch.md
# TODO: Resettare pesi ed optimizert ad ogni KFold, è il metodo giusto? Chiedere forum --- usare torch.no_grad()?
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
from torchvision.models import resnet50
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader,SubsetRandomSampler
#from torch.optim.lr_scheduler import CosineAnnealingLR,OneCycleLR

import numpy as np
from csv import writer
from tqdm import tqdm
from sklearn.model_selection import KFold

In [ ]:
import ipynb
from ipynb.fs.full.utils import fix_seed, train_one_epoch, test_one_epoch, reset_wgts

In [ ]:
SEED = 128
fix_seed(SEED)

In [ ]:
IMGS_PATH = '/media/users/cgambina/Progetto_6/Dati/Immagini'

NUM_EPOCHS = 6
BS = 64 
LR = 1e-4
K = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
splits = KFold(n_splits=K, shuffle=True, random_state=SEED)

In [ ]:
transforms = T.Compose([
    T.RandomChoice(
        [
            T.RandomRotation((-5,5)),
            T.RandomRotation((85,95)),
            T.RandomRotation((175,195)),
            T.RandomRotation((265,275)),
    ]),
    T.ToTensor(),
    T.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
dataset = ImageFolder(IMGS_PATH, transforms)
criterion = nn.BCEWithLogitsLoss() 

net = resnet50(weights='ResNet50_Weights.IMAGENET1K_V1').to(device)

n_filters = net.fc.in_features
net.fc = nn.Linear(n_filters, 1).to(device) 

OPTIMIZER = optim.Adam(net.parameters(), lr=LR)  
SCHEDULER = None

In [ ]:
tot_exps = 15
final_test_loss = []
final_test_acc = []

for i in tqdm(range(tot_exps)):
    epoch_train_loss = []; epoch_train_acc = []
    epoch_test_loss = []; epoch_test_acc = []
    
    SEED += 3
    fix_seed(SEED)
    
    for fold, (train_idx,test_idx) in enumerate(splits.split(np.arange(len(dataset)))):
        
        train_sampler = SubsetRandomSampler(train_idx)
        test_sampler = SubsetRandomSampler(test_idx)
        train_loader = DataLoader(dataset, batch_size=BS, sampler=train_sampler)
        test_loader = DataLoader(dataset, batch_size=BS, sampler=test_sampler)
        
        # Reset parametri
        net.apply(reset_wgts)
        optimizer = OPTIMIZER 
        scheduler = SCHEDULER
    
        for epoch in range(NUM_EPOCHS):
            train_loss, train_correct = train_one_epoch(net,device,train_loader,criterion,optimizer,scheduler)
            test_loss, test_correct = test_one_epoch(net,device,test_loader,criterion)
    
            train_loss = train_loss / len(train_loader.sampler)
            train_acc = train_correct / len(train_loader.sampler) * 100
            test_loss = test_loss / len(test_loader.sampler)
            test_acc = test_correct / len(test_loader.sampler) * 100
              
        epoch_train_loss.append(train_loss); epoch_train_acc.append(train_acc)  
        epoch_test_loss.append(test_loss); epoch_test_acc.append(test_acc)
    
    final_test_acc.append(epoch_train_acc)
    final_test_loss.append(epoch_train_loss)

In [ ]:
avg_test_loss = np.mean(final_test_loss)
avg_test_acc = np.mean(final_test_acc)

std_test_loss = np.std(final_test_loss)
std_test_acc = np.std(final_test_acc)

print(f'Media Test Loss: {avg_test_loss:.4f}')
print(f'Media Test Accuracy: {avg_test_acc:.3f}')

print(f'Std Test Loss: {std_test_loss:.4f}')
print(f'Std Test Accuracy: {std_test_acc:.3f}')

In [ ]:
param_list = [NUM_EPOCHS, BS, LR, SCHEDULER, avg_test_loss, avg_test_acc, std_test_loss, std_test_acc, 'ImageNet']

with open('Results.csv', 'a') as f_object:
    writer_object = writer(f_object)
    writer_object.writerow(param_list)
    f_object.close()

In [ ]:
'''
# Visualizzazione risultati migliori
import pandas as pd

header_list = ['Epoche', 'Batch-size', 'Learning-rate', 'Scheduler', 'Loss', 
               'Accuracy', 'Std-Loss', 'Std-Accuracy', 'Normalization']
df = pd.read_csv('Results.csv', header=None)
df.columns = header_list

df.sort_values('Accuracy', ascending=False).head(10)
'''